# TOP

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib widget

In [2]:
cdata = pd.read_csv('games-features-metacritic-new.csv')
owners = pd.read_csv('steamspy_owners.csv')
scharts = pd.read_csv('Long_GamesData.csv')
usrscore = pd.read_csv('steamspy_userscore.csv')
# meta = pd.read_csv('steamspy_metascore.csv')



# meta = meta.rename(columns={'0':'MetaScore'})
usrscore = usrscore.rename(columns={'0':'UserScore'})
owners = owners.rename(columns={'0':'Owners'})
scharts = scharts.rename(columns={'App_id':'QueryID'})
scharts = scharts[scharts.Avg_Players==1.0]

s1 = pd.merge(cdata, owners, how='inner', on=['QueryID'])
s1 = pd.merge(s1, usrscore, how='inner', on=['QueryID'])

s1 = s1[s1.Owners>0]
# s1 = s1[s1.Metacritic>0]
s1 = s1[~s1.IsFree]
s1 = s1[s1[['PriceInitial', 'PriceFinal']].mean(axis=1)>0.]

plt.style.use('dark_background')

fig, ax = plt.subplots(figsize=(10,6))
fig.subplots_adjust(bottom=0.2)
genres = [i for i in s1.columns if 'Genre' in i]

total = 0
for i,g in enumerate(genres):
    ax.bar(i, s1[g].sum())
    total += s1[g].sum()
    
ax.set_xticks([i for i in range(len(genres))])
_ = ax.set_xticklabels([i.split('Is')[1] for i in genres], rotation=45)
ax.set_xlabel('Genre')
ax.set_ylabel('Games')

ax.set_title('Genre distribution for paid games with Metacritic scores')


s1 = s1[['QueryID', 'QueryName', 'ReleaseDate', 'Metacritic', 'Owners', 'RecommendationCount', 'PackageCount', 'MovieCount', 
        'ScreenshotCount', 'DemoCount', 'AchievementCount', 'PriceInitial', 'PriceFinal', 'UserScore']]

s1['PriceMean'] = s1[['PriceInitial', 'PriceFinal']].mean(axis=1)
s1['logOwners'] = np.log10(s1['Owners'])
s1['Revenue'] = s1['Owners']*s1['PriceMean']
s1['logRevenue'] = np.log10(s1['Revenue'])



s1 = s1[s1['ReleaseDate'] != ' ']
s1 = s1.replace({'ReleaseDate':'Spring 2017'}, '2017-03-01')
s1 = s1.replace({'ReleaseDate':'Early 2017'}, '2017-03-01')

s1['ReleaseDate'] = s1['ReleaseDate'].astype('datetime64')
s1['TodaysDate'] = datetime(2021, 5, 20)
s1['Age'] = ((s1.TodaysDate - s1.ReleaseDate)/np.timedelta64(1, 'M'))
s1 = s1.drop(labels=['ReleaseDate', 'TodaysDate'], axis=1)



s1['MonthlyOwners'] = s1.Owners/s1.Age
s1['logMonthlyOwners'] = np.log(s1.MonthlyOwners)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/kj/miniconda3/envs/erdos/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [3]:
%matplotlib widget
plt.style.use('dark_background')

axes = pd.plotting.scatter_matrix(s1[['Age', 'Metacritic', 'PackageCount', 'MovieCount', 'ScreenshotCount', 'DemoCount', 'AchievementCount', 
                                      'PriceMean', 'Owners', 'logOwners', 'Revenue', 'logRevenue', 'MonthlyOwners', 'logMonthlyOwners', 
                                     'UserScore']], figsize=(10,10))

[a.set_xlabel(a.get_xlabel(), rotation=90) for a in axes.flatten()]
_ = [a.set_ylabel(a.get_ylabel(), rotation=0) for a in axes.flatten()]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Figures for slides

In [28]:
(42, 58, 84)/100

TypeError: unsupported operand type(s) for /: 'tuple' and 'int'

In [45]:
fig.get_facecolor()

(0.0, 0.5019607843137255, 0.0, 1.0)

In [51]:
plt.style.use('dark_background')

fig, ax = plt.subplots(figsize=(10,8))
s = ax.scatter(s1['Age']/12.0, s1.logOwners, c=s1.Metacritic, alpha=0.5, cmap='YlOrRd')
ax.set_xlabel('Age of game (Years)', fontsize=16)
ax.set_ylabel('Steamspy\'s Game Owners Estimate(log10)', fontsize=16)
c = fig.colorbar(s)
c.ax.set_ylabel("Metacritic Score", fontsize=16)
# ax.set_title('Uneven ownership of paid, metacritic-reviewed games')
ax.set_xticks([i for i in range(0,25,2)])
ax.grid()


rgb = [22, 34, 54, 255]
rgb = [i/255. for i in rgb]

fig.set_facecolor(rgb)
ax.set_facecolor(rgb)


<ipython-input-51-9cfaa6fcb58a>:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(10,8))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
fig, ax = plt.subplots()

ax.scatter(s1['Metacritic'][s1['Metacritic']>0], s1['UserScore'][s1['Metacritic']>0], cmap='YlOrRd')
ax.set_xlabel('Metacritic Score')
ax.set_ylabel('Steamspy\'s User Score')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, "Steamspy's User Score")

# Dataset

I'll only focus on these features - 'PackageCount', 'MovieCount', 'ScreenshotCount', 'AchievementCount', 'PriceMean'
This is based on the theory that these represent how games are presented on Steam. So, my goal is to predict the number of mohtly owners given how many packages that game is in, how many movies are presented on the game's steam page, etc. By using monthly owners, I'm assuming that ownership scales with number of months, thus discarding information about older and popular games which have been dormant for a few years.

In [67]:
from sklearn.preprocessing import MinMaxScaler

ind_vars = ['PackageCount', 'MovieCount', 'ScreenshotCount', 'AchievementCount', 'PriceMean']
random_state = 404

X = s1[ind_vars+['MonthlyOwners', 'logOwners']].copy()

X = X[s1.Age>6] # This is to remove noisy Age estimates (see s1[s1.MonthlyOwners>1e6])

In [68]:
ax = X.hist(['MonthlyOwners'], bins=50)
ax[0][0].set_yscale('log')
ax[0][0].set_xscale('log')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Since there are only a few games above 400,000 monthly owners, we'll skip those.

In [69]:
X = X[X.MonthlyOwners<100000]
print(X.shape)

(34337, 7)


Finally we scale the data, and create a training and test data split. Before we do the split, lets check the histogram of values to make sure we don't need to stratify based on a feature or predictor. Our predictor will be monthly game owners (or maybe log of it).

In [70]:
axes = X.hist(bins=50)
for ax in axes.flatten():
    ax.set_yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Error to beat

In [72]:
print('MonthlyOwners baseline error {}'.format(np.abs(X.MonthlyOwners-X.MonthlyOwners.mean()).mean()))
print('logOwners baseline error {}'.format(np.abs(X.logOwners-X.logOwners.mean()).mean()))

scaler = MinMaxScaler()
scaler.fit(X)

X_scaled = X.copy()
X_scaled[X_scaled.columns] = scaler.transform(X)

data_train = X_scaled.sample(frac=.75, random_state=random_state)
data_test = X_scaled.drop(data_train.index)

print(X.shape)

MonthlyOwners baseline error 2079.0429109266684
logOwners baseline error 0.6753325334045595
(34337, 7)


In [73]:
def reverse_scale(x, scaler_ind = -1):
    x2 = np.zeros((x.shape[0], scaler.n_features_in_))
    x2[:,scaler_ind] = x
    x = scaler.inverse_transform(x2)[:,scaler_ind]
    return x

In [74]:
axes1 = data_train.hist(bins=50)
for ax in axes1.flatten():
    ax.set_yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
axes = data_test.hist(bins=50)
for ax, ax1 in zip(axes.flatten(), axes1.flatten()):
    ax.set_yscale('log')
    ax.set_xlim(ax1.get_xlim())
    ax.set_ylim(ax1.get_ylim())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Models 

## PCA

Let's look at how marketing a game on Steam affects the game's selling ability. We will try to predict the log10(owners), since it seems linearly correlated to other varibles. Using revenue also doesn't make sense since it is owners x priceMean, and we are 


In [239]:
from sklearn.decomposition import PCA

p = PCA()
out = p.fit_transform(X_scaled[ind_vars])

fig, ax = plt.subplots()
ax.plot(np.cumsum(p.explained_variance_ratio_))
ax.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [240]:
X.MonthlyOwners.min()

0.40180693069306933

In [242]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

_ = ax.scatter(out[:,0], out[:,1], out[:,2], c = np.log10(X.MonthlyOwners), cmap='YlOrRd', alpha=0.7, s=10)
fig.colorbar(_)
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('PCA 3')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'PCA 3')

In [243]:
fig, ax = plt.subplots(figsize=(10,4))

for i in range(p.n_components_):
    for pi in range(p.n_components_):
        ax.bar((i-0.4+pi*(0.8/p.n_components_)), p.components_[i, pi], width=(0.7/p.n_components_), label='Angle %i'%i)
        ax.set_prop_cycle(None)

ax.set_xlabel('Leading PCA Components ')
ax.set_ylabel('Proportion along PC direction')
# ax.legend()
ax.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Linear/Ridge/Lasso regressions

In [223]:
def powerset_no_empty(s):
    power_set = []
    x = len(s)
    for i in range(1 << x):
        power_set.append([s[j] for j in range(x) if (i & (1 << j))])            
    return power_set[1:]

possible_features = powerset_no_empty(ind_vars)

In [228]:
fig, ax = plt.subplots()
ax.errorbar(x=np.arange(rMSE_test.shape[1]), y=rMSE_test.mean(axis=0), yerr=rMSEs.std(axis=0))
ax.set_title('Median %0.02f'%(np.nanmedian(np.log10(np.abs((pred-y_test)/y_test)))))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Median 0.59')

In [238]:
err = np.abs(pred-y_test)
logerr = np.log10(err/y_test)
fig, ax = plt.subplots()
_ = ax.hist(logerr, bins=100, edgecolor='k')
ax.set_title('Mean %0.2f, Median %0.2f'%(np.abs(logerr).mean(), np.median(np.abs(logerr))))
# ax.set_ylim([0,10])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Mean 0.80, Median 0.72')

In [233]:
print('Last regression features were ', possible_features[-1])
print()
print('Intercept {}, Coefficients {}\n'.format(clone_reg.intercept_, clone_reg.coef_))

Last regression features were  ['PackageCount', 'MovieCount', 'ScreenshotCount', 'AchievementCount', 'PriceMean']

Intercept -0.007518874790269194, Coefficients [0.32132327 0.16576787 0.05786532 0.02982476 0.5470826 ]



## Decision Tree Regressions

In [295]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.base import clone

In [296]:
random_state = 404
n_kfolds = 5

rMSEs = np.empty((n_kfolds,len(possible_features)))
rMSE_test = np.empty((n_kfolds, len(possible_features)))

reg = DecisionTreeRegressor()

kfold = KFold(n_splits = n_kfolds, shuffle = True, random_state = random_state)

i = 0
for train_index, test_index in kfold.split(data_train.index):
    ## For each possible model
    for j in range(len(possible_features)):
        ## get X and y
        Xin, yin = np.array(data_train[possible_features[j]]), np.array(data_train['MonthlyOwners'])


        # Get the cv train test split
        X_train, X_test = Xin[train_index], Xin[test_index]
        y_train, y_test = yin[train_index], yin[test_index]
        
        # Cloning the regression makes a fresh regression 
        # model for each run
        clone_reg = clone(reg)
        
        # fit the model
        clone_reg.fit(X_train,y_train)

        pred = reverse_scale(clone_reg.predict(X_test), scaler_ind=-2)
        y_test = reverse_scale(y_test, scaler_ind=-2)
        err = np.abs(pred-y_test)
        y_test = y_test[err>0]
        pred = pred[err>0]        
        err = err[err>0]        
        rMSEs[i,j] = np.nanmean(np.log10(err/y_test))

        pred = reverse_scale(clone_reg.predict(data_test[possible_features[j]].values), scaler_ind=-2)
        y_test = reverse_scale(data_test['MonthlyOwners'], scaler_ind=-2)
        err = np.abs(pred-y_test)
        y_test = y_test[err>0]
        pred = pred[err>0]
        err = err[err>0]
        rMSE_test[i, j] = np.nanmean(np.log10(err/y_test))
    
    ## We'll now move to the next split
    i = i + 1

fig, ax = plt.subplots()
ax.errorbar(x=np.arange(rMSEs.shape[1]), y=rMSEs.mean(axis=0), yerr=rMSEs.std(axis=0))



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [304]:
fig, ax = plt.subplots()
plt.set_cmap('YlOrRd')
ax.scatter(np.arange(len(pred)), y_test,  alpha=0.6)
ax.scatter(np.arange(len(pred)), pred,alpha=0.6, marker='x')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [311]:
logerr = np.log10(err/y_test)
fig, ax = plt.subplots()
_ = ax.hist(logerr, bins=100, edgecolor='k')
ax.set_title('Mean %0.2f, Median %0.2f'%(np.abs(logerr).mean(), np.median(np.abs(logerr))))
# ax.set_ylim([0,10])
ax.set_xlabel()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Mean 0.60, Median 0.45')

In [293]:
from sklearn import tree

text_representation = tree.export_text(clone_reg)
print(text_representation)

|--- feature_4 <= 0.02
|   |--- feature_4 <= 0.01
|   |   |--- feature_0 <= 0.03
|   |   |   |--- feature_3 <= 0.01
|   |   |   |   |--- feature_2 <= 0.04
|   |   |   |   |   |--- feature_1 <= 0.03
|   |   |   |   |   |   |--- feature_4 <= 0.00
|   |   |   |   |   |   |   |--- feature_2 <= 0.04
|   |   |   |   |   |   |   |   |--- feature_4 <= 0.00
|   |   |   |   |   |   |   |   |   |--- feature_4 <= 0.00
|   |   |   |   |   |   |   |   |   |   |--- feature_4 <= 0.00
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 10
|   |   |   |   |   |   |   |   |   |   |--- feature_4 >  0.00
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 6
|   |   |   |   |   |   |   |   |   |--- feature_4 >  0.00
|   |   |   |   |   |   |   |   |   |   |--- value: [0.10]
|   |   |   |   |   |   |   |   |--- feature_4 >  0.00
|   |   |   |   |   |   |   |   |   |--- feature_4 <= 0.00
|   |   |   |   |   |   |   |   |   |   |--- feature_2 <= 0.03
|   |   |   | 

## Random Forest

In [250]:
from sklearn.ensemble import RandomForestRegressor
random_state = 404
n_kfolds = 5

rMSEs = np.empty((n_kfolds,len(possible_features)))
rMSE_test = np.empty((n_kfolds, len(possible_features)))

reg = RandomForestRegressor(random_state = random_state)

kfold = KFold(n_splits = n_kfolds, shuffle = True, random_state = random_state)

i = 0
for train_index, test_index in kfold.split(data_train.index):
    ## For each possible model
    for j in range(len(possible_features[:])):
        print(i, j)
        ## get X and y
        Xin, yin = np.array(data_train[possible_features[j]]), np.array(data_train['MonthlyOwners'])


        # Get the cv train test split
        X_train, X_test = Xin[train_index], Xin[test_index]
        y_train, y_test = yin[train_index], yin[test_index]
        
        # Cloning the regression makes a fresh regression 
        # model for each run
        clone_reg = clone(reg)
        
        # fit the model
        clone_reg.fit(X_train,y_train)

        pred = reverse_scale(clone_reg.predict(X_test), scaler_ind=-2)
        y_test = reverse_scale(y_test, scaler_ind=-2)
        err = np.abs(pred-y_test)
        y_test = y_test[err>0]
        pred = pred[err>0]        
        err = err[err>0]        
        rMSEs[i,j] = np.nanmean(np.log10(err/y_test))

        pred = reverse_scale(clone_reg.predict(data_test[possible_features[j]].values), scaler_ind=-2)
        y_test = reverse_scale(data_test['MonthlyOwners'], scaler_ind=-2)
        err = np.abs(pred-y_test)
        y_test = y_test[err>0]
        pred = pred[err>0]
        err = err[err>0]
        rMSE_test[i, j] = np.nanmean(np.log10(err/y_test))
    
    ## We'll now move to the next split
    i = i + 1

fig, ax = plt.subplots()
ax.errorbar(x=np.arange(rMSEs.shape[1]), y=rMSEs.mean(axis=0), yerr=rMSEs.std(axis=0))



0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23


KeyboardInterrupt: 

In [ ]:
logerr = np.log10(err/y_test)
fig, ax = plt.subplots()
_ = ax.hist(logerr, bins=100, edgecolor='k')
ax.set_title('Mean %0.2f, Median %0.2f'%(np.abs(logerr).mean(), np.median(np.abs(logerr))))
# ax.set_ylim([0,10])

## tSNE

In [76]:
from sklearn.manifold import TSNE
from umap import UMAP
from sklearn.cluster import KMeans

inp = X_scaled[ind_vars]

In [109]:
inp.shape

(34337, 5)

In [63]:
fig, axes = plt.subplots(3, 2, figsize=(12,16))
for n_neighbors, ax in zip([10, 20, 35, 64, 128, 256], axes.flatten()):
    u = UMAP(n_neighbors=n_neighbors, verbose=True)
    yu = u.fit_transform(inp)#, target)
    s = ax.scatter(yu[:,0], yu[:,1], s=2, c=data_train.logOwners, cmap='YlOrRd')
    ax.set_title('UMAP %i'%n_neighbors)

    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

UMAP(dens_frac=0.0, dens_lambda=0.0, n_neighbors=10, verbose=True)
Construct fuzzy simplicial set
Thu May 27 16:13:23 2021 Finding Nearest Neighbors
Thu May 27 16:13:23 2021 Building RP forest with 12 trees
Thu May 27 16:13:23 2021 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	Stopping threshold met -- exiting after 3 iterations
Thu May 27 16:13:23 2021 Finished Nearest Neighbor Search
Thu May 27 16:13:23 2021 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Thu May 27 16:13:44 2021 Finished embedding
UMAP(dens_frac=0.0, dens_lambda=0.0, n_neighbors=20, verbose=True)
Construct fuzzy simplicial set
Thu May 27 16:13:44 2021 Finding Nearest Neighbors
Thu May 27 16:13:44 2021 Building RP forest with 12 

In [100]:
u = UMAP(n_neighbors=16, verbose=True)
yu = u.fit_transform(inp)#, target)

UMAP(dens_frac=0.0, dens_lambda=0.0, n_neighbors=16, verbose=True)
Construct fuzzy simplicial set
Fri May 28 16:12:34 2021 Finding Nearest Neighbors
Fri May 28 16:12:34 2021 Building RP forest with 14 trees
Fri May 28 16:12:34 2021 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	Stopping threshold met -- exiting after 2 iterations
Fri May 28 16:12:35 2021 Finished Nearest Neighbor Search
Fri May 28 16:12:35 2021 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Fri May 28 16:13:30 2021 Finished embedding


In [101]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

s = ax.scatter(yu[:,0], yu[:,1], s=2, c=np.log10(X.MonthlyOwners), cmap='YlOrRd')

c = fig.colorbar(s, ax = ax)
ax.axis('off')

rgb = [22, 34, 54, 255]
rgb = [i/255. for i in rgb]

fig.set_facecolor(rgb)
ax.set_facecolor(rgb)

c.ax.set_ylabel(r"log$_{10}$ monthly sales", fontsize=14)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'log$_{10}$ monthly sales')

In [253]:
t = TSNE(verbose=1, perplexity=32)
y = t.fit_transform(inp)

[t-SNE] Computing 97 nearest neighbors...
[t-SNE] Indexed 34337 samples in 0.627s...
[t-SNE] Computed neighbors for 34337 samples in 2.296s...
[t-SNE] Computed conditional probabilities for sample 1000 / 34337
[t-SNE] Computed conditional probabilities for sample 2000 / 34337
[t-SNE] Computed conditional probabilities for sample 3000 / 34337
[t-SNE] Computed conditional probabilities for sample 4000 / 34337
[t-SNE] Computed conditional probabilities for sample 5000 / 34337
[t-SNE] Computed conditional probabilities for sample 6000 / 34337
[t-SNE] Computed conditional probabilities for sample 7000 / 34337
[t-SNE] Computed conditional probabilities for sample 8000 / 34337
[t-SNE] Computed conditional probabilities for sample 9000 / 34337
[t-SNE] Computed conditional probabilities for sample 10000 / 34337
[t-SNE] Computed conditional probabilities for sample 11000 / 34337
[t-SNE] Computed conditional probabilities for sample 12000 / 34337
[t-SNE] Computed conditional probabilities for sam

In [93]:
fig, axes = plt.subplots(1, 2, figsize=(10,5))

ax = axes[0]
s = ax.scatter(y[:,0], y[:,1], s=2, c=X.logOwners, cmap='YlOrRd', )
ax.set_title('TSNE')
fig.colorbar(s, ax = ax)

ax = axes[1]
s = ax.scatter(yu[:,0], yu[:,1], s=2, c=X.logOwners, cmap='YlOrRd')
ax.set_title('UMAP')
fig.colorbar(s, ax = ax)
ax.axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'y' is not defined

In [275]:
# import pickle

# with open('kanishk.model', 'wb') as f:
#     pickle.dump({'umap':u, 'kmeans':k, 'cluster_nos':yk, 'scaler':scaler, 'cluster_means':np.array(cluster_means), 
#                 'embeddings':yu, 'owners':X.Owners.values}, f)

In [102]:
klustery = yu

In [116]:
k = KMeans(n_clusters=60)
yk = k.fit_predict(klustery)

In [117]:
cluster_means = []
owners = reverse_scale(X_scaled.values[:,-2], scaler_ind=-2)
for i in range(k.n_clusters):
    cluster_means.append(np.median(owners[np.where(yk==i)[0]]))
    print('Cluster %i Mean Owners '%(i) + "{:,.0f}".format(cluster_means[-1]))
    


Cluster 0 Mean Owners 505
Cluster 1 Mean Owners 172
Cluster 2 Mean Owners 134
Cluster 3 Mean Owners 105
Cluster 4 Mean Owners 199
Cluster 5 Mean Owners 112
Cluster 6 Mean Owners 343
Cluster 7 Mean Owners 166
Cluster 8 Mean Owners 124
Cluster 9 Mean Owners 105
Cluster 10 Mean Owners 136
Cluster 11 Mean Owners 149
Cluster 12 Mean Owners 192
Cluster 13 Mean Owners 254
Cluster 14 Mean Owners 142
Cluster 15 Mean Owners 388
Cluster 16 Mean Owners 135
Cluster 17 Mean Owners 116
Cluster 18 Mean Owners 192
Cluster 19 Mean Owners 156
Cluster 20 Mean Owners 97
Cluster 21 Mean Owners 118
Cluster 22 Mean Owners 276
Cluster 23 Mean Owners 136
Cluster 24 Mean Owners 124
Cluster 25 Mean Owners 130
Cluster 26 Mean Owners 218
Cluster 27 Mean Owners 189
Cluster 28 Mean Owners 150
Cluster 29 Mean Owners 219
Cluster 30 Mean Owners 97
Cluster 31 Mean Owners 92
Cluster 32 Mean Owners 143
Cluster 33 Mean Owners 99
Cluster 34 Mean Owners 194
Cluster 35 Mean Owners 57
Cluster 36 Mean Owners 93
Cluster 37 Mean O

In [118]:
fig, axes = plt.subplots(3, 2, figsize=(10,15))

rgb = [22, 34, 54, 255]
rgb = [i/255. for i in rgb]

fig.set_facecolor(rgb)

c.ax.set_ylabel(r"log$_{10}$ monthly sales", fontsize=14)

cmap = plt.matplotlib.colors.ListedColormap(["navy", "crimson", "limegreen", "gold", "gray"])

ax = axes[0][1]
s = ax.scatter(klustery[:,0], klustery[:,1], s=2, c=yk, cmap='tab20c' )
fig.colorbar(s, ax=ax, ticks=np.arange(0,k.n_clusters))
ax.set_title('TSNE kMeans clusters')
ax.axis('off')
ax.set_facecolor(rgb)

ax = axes[1][0]
s = ax.scatter(klustery[:,0], klustery[:,1], s=2, c=np.log10(cluster_means)[yk], cmap='YlOrRd' )
fig.colorbar(s, ax=ax,)# ticks=np.arange(0,k.n_clusters))
# ax.set_title('TSNE kMeans Avg Owners')
ax.axis('off')

ax = axes[0][0]
s = ax.scatter(klustery[:,0], klustery[:,1], s=2, c=np.log10(owners), cmap='YlOrRd')
ax.set_title('TSNE log Owners')
fig.colorbar(s, ax=ax)


all_errs =np.abs(np.array(cluster_means)[yk]-owners)*100/owners 
ax = axes[1,1]
ax.plot(all_errs)
ax.set_title('Mean %0.02f, median %0.02f'%(all_errs.mean(), np.median(all_errs)))

all_errs = np.log10(np.array(cluster_means)[yk]/owners)
ax = axes[2,0]
ax.plot(all_errs)
ax.set_title('Mean %0.02f, median %0.02f'%(np.abs(all_errs).mean(), np.median(np.abs(all_errs))))


axes[2,1].axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 1.0, 0.0, 1.0)

In [276]:
testinp = data_test[ind_vars]
yu_test = u.transform(testinp)
yk_test = k.predict(yu_test)




Fri May 28 11:10:23 2021 Worst tree score: 0.84943356
Fri May 28 11:10:23 2021 Mean tree score: 0.87550081
Fri May 28 11:10:23 2021 Best tree score: 0.97841978
Fri May 28 11:10:25 2021 Forward diversification reduced edges from 686740 to 437842
Fri May 28 11:10:27 2021 Reverse diversification reduced edges from 437842 to 437824


/Users/kj/miniconda3/envs/erdos/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Fri May 28 11:10:31 2021 Degree pruning reduced edges from 483842 to 483831
Fri May 28 11:10:31 2021 Resorting data and graph based on tree order
Fri May 28 11:10:31 2021 Compressing index by removing unneeded attributes
Fri May 28 11:10:31 2021 Building and compiling search function
	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs


In [277]:
pred_owners = np.array(cluster_means)[yk_test]
test_owners = reverse_scale(data_test.values[:,-2], scaler_ind=-2)


In [278]:
fig, axes = plt.subplots(3, 2, figsize=(10,15))


ax = axes[0,1]
s = ax.scatter(yu_test[:,0], yu_test[:,1], s=2, c=yk_test, cmap='tab20c' )
fig.colorbar(s, ax=ax, ticks=np.arange(0,k.n_clusters))
ax.set_title('TSNE kMeans clusters')

ax = axes[0,0]
s = ax.scatter(yu_test[:,0], yu_test[:,1], s=2, c=np.log10(test_owners), cmap='YlOrRd')
ax.set_title('TSNE log Owners')
fig.colorbar(s, ax=ax)

ax = axes[1,0]
s = ax.scatter(yu_test[:,0], yu_test[:,1], s=2, c=np.log10(pred_owners), cmap='YlOrRd')
ax.set_title('TSNE prediction log Owners')
fig.colorbar(s, ax=ax)


all_errs =np.abs(pred_owners-test_owners)*100/test_owners 
ax = axes[1,1]
ax.plot(all_errs)
ax.set_title('Mean %0.02f, median %0.02f'%(all_errs.mean(), np.median(all_errs)))

all_errs = np.log10(pred_owners/test_owners)
ax = axes[2,0]
ax.plot(all_errs)
ax.set_title('Mean %0.02f, median %0.02f'%(np.abs(all_errs).mean(), np.median(np.abs(all_errs))))


axes[2,1].axis('off')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 1.0, 0.0, 1.0)

## Neural Network

In [279]:
import keras

In [284]:
inp = data_train[ind_vars].values

In [285]:
target = data_train['MonthlyOwners'].values

# target = reverse_scale(data_train['Owners'].values,scaler_ind=-2)
print(target.shape)

(25753,)


In [287]:
valx = data_test[ind_vars].values
valy = data_test.MonthlyOwners.values

In [288]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(5,)))
model.add(keras.layers.Dense(16, activation='selu'))
model.add(keras.layers.Dense(1, activation='selu'))
model.compile('adam', 'mse')
model.summary()

hist = model.fit(inp, target, batch_size=16, verbose=1, epochs=200, validation_data=(valx, valy), shuffle=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                96        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
1610/1610 [==============================] - 11s 921us/step - loss: 0.0033 - val_loss: 0.0025
Epoch 2/200
1610/1610 [==============================] - 1s 767us/step - loss: 0.0032 - val_loss: 0.0025
Epoch 3/200
1610/1610 [==============================] - 1s 808us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 4/200
1610/1610 [==============================] - 1s 776us/step - loss: 0.0031 - val_loss: 0.0026
Epoch 5/200
1610/1610 [==============================] - 1s 826us/step - loss: 0.0030 - val_loss: 0

In [34]:
fig, ax = plt.subplots()
ax.plot(hist.history['loss'], 'b-')
ax.plot(hist.history['val_loss'], 'r-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'hist' is not defined

In [35]:
pred = model.predict(valx)

pred = reverse_scale(pred.flatten(), scaler_ind=-2)

test_target = reverse_scale(valy, scaler_ind=-2)

In [39]:
fig, axes = plt.subplots(2, 1, figsize=(8, 4))
errs = (pred-test_target)*100/test_target
ax = axes[0]
ax.plot(errs)
ax.set_title('Mean %0.02i, Median %0.02i'%(errs.mean(), np.median(errs)))

ax = axes[1]
errs = np.log10((pred-test_target)/test_target)
ax.plot(errs)
ax.set_title('Mean %0.02i, Median %0.02i'%(errs.nanmean(), np.nanmedian(errs)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-39-42cc8fd82a2e>:8: RuntimeWarning: invalid value encountered in log10
  errs = np.log10((pred-test_target)/test_target)


AttributeError: 'numpy.ndarray' object has no attribute 'nanmean'

# Timeseries

In [52]:
ts = pd.merge(s1, scharts, how='inner', on=['QueryID'])


In [56]:
ts.loc[0], ts.loc[2]

,QueryID,QueryName,Metacritic,Owners,RecommendationCount,PackageCount,MovieCount,ScreenshotCount,DemoCount,AchievementCount,...,April_2013,March_2013,February_2013,January_2013,December_2012,November_2012,October_2012,September_2012,August_2012,July_2012
0,10,Counter-Strike,88,18429000.0,68991,1,0,13,0,0,...,21204.46,24139.15,28378.42,34814.47,31996.07,29669.97,28836.29,29432.56,33095.8,34139.2
1,20,Team Fortress Classic,-1,4137000.0,2439,1,0,5,0,0,...,73.58,77.73,82.32,89.0,87.79,78.58,77.51,83.83,93.03,98.17
2,30,Day of Defeat,79,6159000.0,2319,1,0,5,0,0,...,217.9,297.55,318.41,319.62,318.08,303.99,297.0,289.59,304.47,331.84
3,40,Deathmatch Classic,-1,5942000.0,888,1,0,4,0,0,...,4.64,5.78,7.11,6.91,6.72,6.18,5.56,6.26,7.41,8.61
4,50,Half-Life: Opposing Force,-1,8119000.0,2934,1,0,5,0,0,...,43.04,39.92,43.15,50.2,47.06,38.63,41.3,56.26,55.77,79.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8237,431950,Age of Fear 4: The Iron Killer,-1,4000.0,0,1,2,13,0,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8238,1476760,Amazing Fix,0,12000.0,0,1,1,6,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8239,1040230,Curious Expedition 2,0,42000.0,1056,1,2,12,0,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8240,1012790,Into the Radius VR,0,51000.0,1114,1,3,8,0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
cdata['ReleaseDate']

0                                               Nov 1 2000
1                                               Apr 1 1999
2                                               May 1 2003
3                                               Jun 1 2001
4                                               Nov 1 1999
                               ...                        
57205                                          May 14 2021
57206    coming summer -  Wishlist now - discount at la...
57207                                        Wishlist Now!
57208                                          May 14 2021
57209                                          Apr 16 2020
Name: ReleaseDate, Length: 57210, dtype: object

In [474]:
s1[X.index==1205]

,QueryID,QueryName,Metacritic,Owners,RecommendationCount,PackageCount,MovieCount,ScreenshotCount,DemoCount,AchievementCount,PriceInitial,PriceFinal,PriceMean,logOwners,Revenue,logRevenue,Age,MonthlyOwners,logMonthlyOwners
1205,50100,Sid Meier's Civilization V,90,1000.0,85790,2,5,5,1,287,29.99,29.99,29.99,3.0,29990.0,4.476976,127.936919,7.816352,2.056218


In [475]:

ts[ts.QueryID==50100]

,QueryID,QueryName,Metacritic,Owners,RecommendationCount,PackageCount,MovieCount,ScreenshotCount,DemoCount,AchievementCount,...,April_2013,March_2013,February_2013,January_2013,December_2012,November_2012,October_2012,September_2012,August_2012,July_2012


In [57]:
plt.set_cmap('viridis')
fig, ax = plt.subplots(figsize=(10,5))
fig.subplots_adjust(bottom=0.2)

columnind = np.where(ts.columns=='April_2021')[0][0]
months = ts.columns.values[columnind:]

plotted = 0
for i in range(len(ts)):
    if not cdata[cdata['QueryID']==ts.loc[i]['QueryID']]['GenreIsIndie'].values[0]:
        continue
    vals = ts.loc[i].values[columnind:].astype(float)
    
    if not np.isfinite(vals).any():
        continue
        
    if vals[np.isfinite(vals)].mean()<10000:
        continue
        
    if not np.isfinite(vals).all():
        nanind = np.where(~np.isfinite(vals))[0][0]
    else:
        nanind = len(vals)
    ax.plot(vals[:nanind], label=ts['QueryName'][i])
    plotted += 1
    if plotted > 9:
        break
# fig.legend(loc='upper center', fontsize=7)

# xticks = ax.get_xticks()
# _ = ax.set_xticks([i for i in xticks if i>-1 and int(i)<len(months)])
# _ = ax.set_xticklabels([np.array(months)[::-1][int(i)] for i in xticks if int(i)<len(months) and i>-1], rotation=0)

ax.set_xticks([i for i in range(0, len(months), 8)])
_ = ax.set_xticklabels([np.array(months)[int(i)] for i in range(0, len(months), 8)], rotation=45)

ax.set_ylabel('# Players')
ax.legend(loc='upper center', )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
ax.grid(alpha=0.51)